# MCQ Inference Notebook - Direct llama.cpp Client

This notebook uses `OllamaCppClient` directly for MCQ answering.

## Architecture

```
┌─────────────────────────────────────────────────┐
│  Notebook (run_inference.ipynb)                 │
│  ├─ ask_single_question_cpp()                   │
│  └─ Direct import from ollamacpp_client.py      │
└────────────────────┬────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────┐
│  OllamaCppClient (ollamacpp_client.py)          │
│  ├─ generate_mcq_answer()                       │
│  ├─ _build_mcq_prompt()                         │
│  └─ _parse_mcq_response()                       │
└────────────────────┬────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────┐
│  llama.cpp Server                               │
│  http://127.0.0.1:8080/v1/chat/completions      │
└─────────────────────────────────────────────────┘
```

## Features

✅ **Direct llama.cpp integration** - No FastAPI overhead  
✅ **Optional context support** - Pass context_chunks or None  
✅ **Async/sync wrappers** - Easy to use in notebooks  
✅ **Parallel processing** - Process multiple questions concurrently  
✅ **Flexible modes** - Choose between RAG API or direct generation

## Quick Start

1. **Make sure llama.cpp server is running:**
   ```bash
   # Check if server is running
   curl http://127.0.0.1:8080/health
   ```

2. **Run cell 1** to initialize `OllamaCppClient`

3. **Choose your mode:**
   - **Direct generation (fastest):** Uses current `main()` function
   - **With RAG API:** Use `ask_single_question_with_rag()`

In [ ]:
import os
from pathlib import Path
from src.config import get_settings
from src.services.llamacpp.llamacpp_client import OllamaCppClient

# ./server -m "C:\Users\APC\AppData\Local\llama.cpp\bartowski_SmallThinker-3B-Preview-GGUF_SmallThinker-3B-Preview-Q4_K_M.gguf" -c 16384
API_BASE_URL = "http://localhost:8000"
AIRFLOW_URL = "http://localhost:8080"

# Host & port các service
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432          # hoặc 5433 nếu muốn connect airflow-db
REDIS_HOST = "localhost"
REDIS_PORT = 6379
OPENSEARCH_HOST = "http://localhost:9200"
LLAMACPP_HOST = "http://127.0.0.1:8080"

# Cấu hình RAG
TOP_K = 10
USE_HYBRID = True
TIMEOUT = 180
MAX_WORKERS = 2

# Initialize llama.cpp client
llama_client = OllamaCppClient(
    host=LLAMACPP_HOST,
    model="gpt-like-model",
    temperature=0.3,
    max_response_words=300,
    timeout=TIMEOUT
)

settings = get_settings()
pdf_dir = Path(settings.data.pdf_dir)

pdf_files = list(pdf_dir.glob("*.pdf"))
pdf_paths = [str(f) for f in pdf_files]

print(f"Found {len(pdf_paths)} PDF files:")
print(f"Llama.cpp client initialized at: {LLAMACPP_HOST}")
pdf_paths

Found 58 PDF files:
Llama.cpp client initialized at: http://127.0.0.1:8080


['data\\pdf\\Public_427.pdf',
 'data\\pdf\\Public_428.pdf',
 'data\\pdf\\Public_441.pdf',
 'data\\pdf\\Public_442.pdf',
 'data\\pdf\\Public_444.pdf',
 'data\\pdf\\Public_445.pdf',
 'data\\pdf\\Public_446.pdf',
 'data\\pdf\\Public_447.pdf',
 'data\\pdf\\Public_448.pdf',
 'data\\pdf\\Public_449.pdf',
 'data\\pdf\\Public_465.pdf',
 'data\\pdf\\Public_466.pdf',
 'data\\pdf\\Public_468.pdf',
 'data\\pdf\\Public_469.pdf',
 'data\\pdf\\Public_470.pdf',
 'data\\pdf\\Public_471.pdf',
 'data\\pdf\\Public_472.pdf',
 'data\\pdf\\Public_473.pdf',
 'data\\pdf\\Public_474.pdf',
 'data\\pdf\\Public_624.pdf',
 'data\\pdf\\Public_626.pdf',
 'data\\pdf\\Public_634.pdf',
 'data\\pdf\\Public_635.pdf',
 'data\\pdf\\Public_636.pdf',
 'data\\pdf\\Public_637.pdf',
 'data\\pdf\\Public_638.pdf',
 'data\\pdf\\Public_639.pdf',
 'data\\pdf\\Public_640.pdf',
 'data\\pdf\\Public_641.pdf',
 'data\\pdf\\Public_642.pdf',
 'data\\pdf\\Public_643.pdf',
 'data\\pdf\\Public_644.pdf',
 'data\\pdf\\Public_645.pdf',
 'data\\pd

In [13]:
import requests
import pandas as pd
import time
import json
from pathlib import Path
from typing import Dict, Any, List
from datetime import datetime
import socket
from urllib.parse import urlparse

# Đường dẫn files
DATA_DIR = Path("data")
PDF_DIR = DATA_DIR / "pdf"
QUESTION_FILE = DATA_DIR / "question.csv"
OUTPUT_FILE = DATA_DIR / "answers_mcq.csv"

In [14]:
def print_header(text: str):
    print("\n" + "=" * 80)
    print(f"  {text}")
    print("=" * 80)

def print_step(step_num: int, text: str):
    print(f"\n{'─' * 80}")
    print(f"📌 BƯỚC {step_num}: {text}")
    print(f"{'─' * 80}")

def print_step(step_num: int, text: str):
    print(f"\n{'─' * 80}")
    print(f"📌 BƯỚC {step_num}: {text}")
    print(f"{'─' * 80}")


def load_questions() -> pd.DataFrame:
    """Đọc câu hỏi từ CSV file."""
    print_step(4, "Đọc câu hỏi từ CSV")

    if not QUESTION_FILE.exists():
        print(f"\n❌ File không tồn tại: {QUESTION_FILE}")
        print("Vui lòng tạo file question.csv với format:")
        print("  Question,A,B,C,D,source_folder")
        return None

    try:
        df = pd.read_csv(QUESTION_FILE)
        print(f"\n✅ Đọc thành công {len(df)} câu hỏi")

        # Kiểm tra columns
        required_cols = ["Question", "A", "B", "C", "D"]
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            print(f"❌ Thiếu columns: {missing_cols}")
            return None

        # Hiển thị sample
        print("\n📝 Ví dụ câu hỏi đầu tiên:")
        print(f"   Q: {df.iloc[0]['Question']}...")

        return df

    except Exception as e:
        print(f"❌ Lỗi khi đọc CSV: {e}")
        return None

load_questions().head()


────────────────────────────────────────────────────────────────────────────────
📌 BƯỚC 4: Đọc câu hỏi từ CSV
────────────────────────────────────────────────────────────────────────────────

✅ Đọc thành công 282 câu hỏi

📝 Ví dụ câu hỏi đầu tiên:
   Q: Tuổi thọ trung bình của tấm pin năng lượng mặt trời là bao nhiêu năm?...


,Question,A,B,C,D
0,Tuổi thọ trung bình của tấm pin năng lượng mặt...,20 – 25 năm,10 – 12 năm,5 – 7 năm,10 - 15 năm
1,Biểu tượng nào hiển thị màu lục khi động cơ se...,Biểu tượng chế độ thủ công,Biểu tượng gia nhiệt khoang chứa,Biểu tượng hoạt động của động cơ servo,Biểu tượng báo động
2,"Theo tài liệu Public_656, LED nào sáng khi trố...",Toner LED,Drum LED,Error LED,Ready LED
3,Nút bàn di chuột phải có chức năng gì?,Hoạt động như nút trái chuột ngoài,Hoạt động như nút phải chuột ngoài,Dùng để cuộn trang,Dùng để bật/tắt touchpad
4,Thành phần nào dùng để đặt giới hạn tối đa có ...,Manual Settings Limits,Auto Settings Limits,Mechanical Settings Limits,Servo Movement Alarms


In [15]:
# Option 1: Use RAG API endpoint (retrieval + generation)
# def ask_single_question_with_rag(
#     question: str, options: Dict[str, str], source_folder: str = None
# ) -> Dict[str, Any]:
#     """Trả lời câu hỏi MCQ using full RAG API (hybrid retrieval + llama.cpp generation)."""
#     payload = {"question": question, "options": options, "top_k": TOP_K, "use_hybrid": USE_HYBRID}
#     if source_folder and pd.notna(source_folder):
#         payload["source_folder"] = source_folder
#     try:
#         response = requests.post(f"{API_BASE_URL}/api/v1/ask", json=payload, timeout=TIMEOUT)
#         response.raise_for_status()
#         return response.json()
#     except requests.exceptions.Timeout:
#         print(f"    ⏱️  Timeout (>{TIMEOUT}s)")
#         return {"error": "timeout", "predicted_option": None}
#     except Exception as e:
#         print(f"    ❌ Lỗi: {str(e)[:100]}")
#         return {"error": str(e), "predicted_option": None}


# Option 2: Use OllamaCppClient directly (manual retrieval, just generation)
async def ask_single_question_direct(
    question: str,
    options: Dict[str, str],
    context_chunks: List[Dict] = None
) -> Dict[str, Any]:
    """
    Trả lời câu hỏi MCQ using OllamaCppClient directly.

    Args:
        question: Question text
        options: Dict of options {"A": "...", "B": "...", ...}
        context_chunks: Optional list of retrieved context chunks
                        If None, will answer without context

    Returns:
        Dict with predicted_option, reasoning, confidence, etc.
    """
    try:
        # Use empty context if none provided
        if context_chunks is None:
            context_chunks = []

        # Call llama.cpp client directly
        result = await llama_client.generate_mcq_answer(
            question=question,
            options=options,
            context_chunks=context_chunks
        )

        return result

    except Exception as e:
        print(f"    ❌ Lỗi: {str(e)[:100]}")
        return {
            "error": str(e),
            "predicted_option": None,
            "confidence": "low",
            "reasoning": f"Error: {str(e)}"
        }


# Option 3: Synchronous wrapper for direct client usage
def ask_single_question_cpp(
    question: str,
    options: Dict[str, str],
    source_folder: str = None,
    context_chunks: List[Dict] = None
) -> Dict[str, Any]:
    """
    Synchronous wrapper for OllamaCppClient.
    Can be used with or without context chunks.
    """
    import asyncio

    # Create event loop if needed
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

    # Run async function
    result = loop.run_until_complete(
        ask_single_question_direct(question, options, context_chunks)
    )

    return result

In [16]:
import re
import requests
from IPython.display import Markdown

def retrieve(query, top_k=TOP_K, use_hybrid=USE_HYBRID):
    """Retrieve chunks from OpenSearch API."""
    url = "http://localhost:8000/api/v1/search/"  # Fixed URL (was missing /)
    payload = {
        "query": query,
        "top_k": top_k,
        "use_hybrid": use_hybrid
    }
    try:
        r = requests.post(url, json=payload, timeout=30)
        r.raise_for_status()
        return r.json()
    except requests.RequestException as e:
        print(f"❌ retrieve() error: {e}")
        return {"hits": []}


def extract_doc_id_from_question(question: str) -> str:
    """
    Extract Public_XXX pattern from question text.
    Returns the pattern if found, otherwise None.

    Example: "Question about Public_422" -> "Public_422"
    """
    match = re.search(r'Public_\d+', question)
    return match.group(0) if match else None


def dynamic_retrieve_context(question: str, top_k: int = TOP_K) -> List[Dict]:
    """
    Retrieve top_k chunks for question.
    Auto-adjusts top_k and filtering based on question pattern.

    - If Public_XXX pattern found: top_k=30 and filter by doc_id
    - Otherwise: use default top_k

    Returns list of chunk dicts with metadata.
    """
    # Try to find Public_XXX pattern in question first
    doc_id_prefix = extract_doc_id_from_question(question)

    # ✅ Dynamic top_k adjustment
    effective_top_k = 30 if doc_id_prefix else top_k

    # Get chunks from OpenSearch with adjusted top_k
    response_data = retrieve(question, top_k=effective_top_k, use_hybrid=USE_HYBRID)
    hits = response_data.get('hits', [])

    results = []
    for hit in hits:
        chunk_id = hit.get('chunk_id', '')

        # Filter by doc_id_prefix only if found in question
        if doc_id_prefix:
            if not chunk_id.startswith(doc_id_prefix):
                continue  # Skip chunks that don't match doc_id

        # Build chunk dict with metadata
        results.append({
            'chunk_id': chunk_id,
            'chunk_text': hit.get('chunk_text', ''),
            'section_name': hit.get('section_name'),
            'document_id': hit.get('document_id'),
            'document_file_name': hit.get('document_file_name'),
            'document_title': hit.get('document_title'),
        })

    # ✅ Better logging
    if doc_id_prefix:
        print(f"✅ Found {len(results)} chunks (filtered by {doc_id_prefix}, searched top {effective_top_k})")
    else:
        print(f"✅ Found {len(results)} chunks (default search with top {effective_top_k})")

    return results

In [17]:
def _process_single_question(args: tuple) -> Dict[str, Any]:
    """
    Process a single question for parallel execution.
    Uses OllamaCppClient directly without context retrieval.

    Args:
        args: Tuple of (index, row, q_num, total)

    Returns:
        Result dictionary for the question
    """
    idx, row, q_num, total = args

    progress = (q_num / total) * 100
    print(f"[{q_num}/{total}] ({progress:.1f}%) Q: {row['Question']}")

    options = {"A": str(row["A"]), "B": str(row["B"]), "C": str(row["C"]), "D": str(row["D"])}
    source_folder = row.get("source_folder", None)

    q_start = time.time()

    # Use direct llama.cpp client (no context retrieval)
    # If you want to add context, retrieve it first and pass to context_chunks parameter
    answer_data = ask_single_question_cpp(
        question=row["Question"],
        options=options,
        context_chunks=dynamic_retrieve_context(row["Question"])  # No context - direct generation only
        # context_chunks=[]  # No context - direct generation only
    )

    q_time = time.time() - q_start

    predicted = answer_data.get("predicted_option", "N/A")
    confidence = answer_data.get("confidence", "unknown")
    reasoning = answer_data.get("reasoning", "")
    error = answer_data.get("error", None)

    conf_icon = {"high": "🟢", "medium": "🟡", "low": "🔴"}.get(confidence, "⚪")
    print(f"    → Đáp án: {predicted} {conf_icon} ({q_time:.1f}s)")
    print(f"      Suy luận: {reasoning}")

    result = {
        "question_number": q_num,
        "question": row["Question"],
        "option_A": row["A"],
        "option_B": row["B"],
        "option_C": row["C"],
        "option_D": row["D"],
        "source_folder": source_folder if pd.notna(source_folder) else "",
        "predicted_answer": predicted,
        "confidence": confidence,
        "reasoning": reasoning[:500] if reasoning else "",
        "processing_time_seconds": round(q_time, 2),
        "error": error if error else "",
        "timestamp": datetime.now().isoformat(),
    }

    return result


def answer_all_questions(df: pd.DataFrame, max_workers: int = 4) -> pd.DataFrame:
    """Trả lời tất cả câu hỏi trong dataframe sử dụng parallel processing."""
    from concurrent.futures import ThreadPoolExecutor, as_completed

    print_step(5, "Trả lời câu hỏi (Parallel)")

    results = []
    total = len(df)

    print(f"\n🎯 Bắt đầu trả lời {total} câu hỏi...\n")
    print(f"⚙️  Using {max_workers} parallel workers\n")
    # print(f"🔧 Mode: Direct llama.cpp generation (no context retrieval)\n")

    start_time = time.time()

    # Prepare task arguments
    tasks = [(idx, row, idx + 1, total) for idx, (_, row) in enumerate(df.iterrows())]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(_process_single_question, task): task[0] for task in tasks}

        completed_count = 0
        for future in as_completed(futures):
            try:
                result = future.result(timeout=TIMEOUT + 30)
                results.append(result)
                completed_count += 1

                if completed_count % 5 == 0:
                    print()

            except Exception as e:
                completed_count += 1
                print(f"❌ Error processing question: {str(e)[:100]}")
                results.append({
                    "error": str(e),
                    "predicted_option": None,
                    "question_number": futures[future] + 1
                })

    total_time = time.time() - start_time

    print("\n" + "─" * 80)
    print(f"✅ Hoàn thành! Tổng thời gian: {total_time:.1f}s")
    print(f"⏱️  Trung bình: {total_time / total:.1f}s/câu")
    print(f"⚡ Tốc độ: {total / total_time:.2f} câu/giây")
    print("─" * 80)

    # Sort results by question number
    results.sort(key=lambda x: x.get("question_number", 0))

    return pd.DataFrame(results)

def save_results(results_df: pd.DataFrame):
    """Lưu kết quả ra CSV file."""
    print_step(6, "Lưu kết quả")

    try:
        results_df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

        print(f"\n✅ Đã lưu kết quả vào: {OUTPUT_FILE}")
        print(f"📊 Tổng số câu: {len(results_df)}")

        # Thống kê
        conf_counts = results_df["confidence"].value_counts()
        print("\n📈 Thống kê độ tin cậy:")
        for conf, count in conf_counts.items():
            percentage = (count / len(results_df)) * 100
            print(f"  • {conf}: {count} câu ({percentage:.1f}%)")

        # Kiểm tra errors
        errors = results_df[results_df["error"] != ""]
        if len(errors) > 0:
            print(f"\n⚠️  Có {len(errors)} câu bị lỗi:")
            for idx, row in errors.iterrows():
                print(f"  • Câu {row['question_number']}: {row['error']}")

    except Exception as e:
        print(f"❌ Lỗi khi lưu file: {e}")

def display_summary(results_df: pd.DataFrame):
    """Hiển thị tóm tắt kết quả."""
    print_header("📊 TÓM TẮT KẾT QUẢ")

    total = len(results_df)
    successful = len(results_df[results_df["error"] == ""])

    print(f"\n✅ Tổng số câu: {total}")
    print(f"✅ Trả lời thành công: {successful}")
    print(f"❌ Lỗi: {total - successful}")

    # Top 5 câu có confidence cao
    high_conf = results_df[results_df["confidence"] == "high"].head(20)
    if len(high_conf) > 0:
        print(f"\n🟢 Top {len(high_conf)} câu có độ tin cậy cao:")
        for idx, row in high_conf.iterrows():
            print(
                f"  {row['question_number']}. Đáp án {row['predicted_answer']}: {row['question']}"
            )

    # Câu có confidence thấp
    low_conf = results_df[results_df["confidence"] == "low"]
    if len(low_conf) > 0:
        print(f"\n🔴 {len(low_conf)} câu có độ tin cậy thấp:")
        for idx, row in low_conf.head(20).iterrows():
            print(f"  {row['question_number']}. {row['question']}")

    print("\n" + "=" * 80)

In [ ]:
def main():
    """Hàm chính để chạy toàn bộ pipeline."""
    MAX_WORKERS  = 4
    print_header("🚀 RAG MCQ SYSTEM - AUTOMATIC ANSWERING")
    print(f"\n📅 Bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Step 3: Check indexed documents
    # doc_count = check_indexed_documents()
    # if doc_count == 0:
    #     print("\n❌ Không có documents nào trong hệ thống!")
    #     print("Không thể trả lời câu hỏi. Vui lòng xử lý PDF trước!")
    #     return

    # Step 4: Load questions
    questions_df = load_questions()
    if questions_df is None:
        return

    # print(questions_df)
    # Step 5: Answer all questions
    results_df = answer_all_questions(questions_df, max_workers=MAX_WORKERS)

    # Step 6: Save results
    save_results(results_df)

main()



  🚀 RAG MCQ SYSTEM - AUTOMATIC ANSWERING

📅 Bắt đầu: 2025-11-12 08:31:19

────────────────────────────────────────────────────────────────────────────────
📌 BƯỚC 4: Đọc câu hỏi từ CSV
────────────────────────────────────────────────────────────────────────────────

✅ Đọc thành công 282 câu hỏi

📝 Ví dụ câu hỏi đầu tiên:
   Q: Tuổi thọ trung bình của tấm pin năng lượng mặt trời là bao nhiêu năm?...
                                              Question  \
0    Tuổi thọ trung bình của tấm pin năng lượng mặt...   
1    Biểu tượng nào hiển thị màu lục khi động cơ se...   
2    Theo tài liệu Public_656, LED nào sáng khi trố...   
3               Nút bàn di chuột phải có chức năng gì?   
4    Thành phần nào dùng để đặt giới hạn tối đa có ...   
..                                                 ...   
277  Trong TD635, có tổng cộng bao nhiêu loại đầu d...   
278  Nếu một thợ kim hoàn cần tạo khuôn và thiết kế...   
279  “Bu lông” và “Cupler” trong TD647 có Tiêu chuẩ...   
280  Với hóa đơn đ

Error generating MCQ answer: Server error '500 Internal Server Error' for url 'http://127.0.0.1:8080/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500
Traceback (most recent call last):
  File "d:\VIETTEL_RACE\qa-mcq-rag\mcqaRAG_p4\src\services\ollama\ollamacpp_client.py", line 70, in generate_mcq_answer
    response = await self._generate(prompt)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\VIETTEL_RACE\qa-mcq-rag\mcqaRAG_p4\src\services\ollama\ollamacpp_client.py", line 264, in _generate
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "d:\VIETTEL_RACE\qa-mcq-rag\mcqaRAG_p4\.venv\Lib\site-packages\httpx\_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'http://127.0.0.1:8080/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/

    → Đáp án: None 🔴 (11.9s)
      Suy luận: Failed to generate answer
[8/282] (2.8%) Q: Văn bản Public_675 chủ yếu mô tả về nội dung gì?
    → Đáp án: None 🔴 (12.2s)
      Suy luận: Failed to generate answer
[9/282] (3.2%) Q: Theo cấu trúc bảng trong TD648, các cột thông tin chính gồm những mục nào?

    → Đáp án: None 🔴 (73.2s)
      Suy luận: Failed to generate answer
[10/282] (3.5%) Q: Văn bản Public 449 mô tả yêu cầu gì về dữ liệu nhập mới? Và bước nào cần kích hoạt nếu dữ liệu không đạt chuẩn?
    → Đáp án: None 🔴 (26.0s)
      Suy luận: Failed to generate answer
[11/282] (3.9%) Q: Vít cấy A1-M12×45 được quy định trong tiêu chuẩn nào?
✅ Found 1 chunks (filtered by Public_675, searched top 30)
✅ Found 20 chunks (default search with top 20)
Prompt TOKEN COUNT: 657
Prompt TOKEN COUNT: 13531
✅ Found 20 chunks (default search with top 20)
Prompt TOKEN COUNT: 11490
✅ Found 20 chunks (default search with top 20)
Prompt TOKEN COUNT: 10356


## Usage Examples
$ ollama-server -hf bartowski/SmallThinker-3B-Preview-GGUF

### Option 1: Direct Generation (No Context)
```python
# Simple generation without context - fastest but least accurate
result = ask_single_question_cpp(
    question="What is machine learning?",
    options={"A": "...", "B": "...", "C": "...", "D": "..."},
    context_chunks=None  # No context
)
```

### Option 2: With Manual Context Retrieval
```python
# If you want to retrieve context yourself and then generate
from src.services.opensearch.client import OpenSearchClient
from src.services.embeddings.sentence_transformer import EmbeddingsService

# Initialize retrieval services
opensearch = OpenSearchClient(host=OPENSEARCH_HOST, ...)
embeddings = EmbeddingsService(...)

# Retrieve context
query_embedding = await embeddings.embed_query(question)
chunks = await opensearch.search_hybrid(query, query_embedding, top_k=30)

# Generate with context
result = ask_single_question_cpp(
    question="What is machine learning?",
    options={"A": "...", "B": "...", "C": "...", "D": "..."},
    context_chunks=chunks  # Retrieved context
)
```

### Option 3: Full RAG Pipeline (Use API)
```python
# Let the API handle everything (retrieval + generation)
result = ask_single_question_with_rag(
    question="What is machine learning?",
    options={"A": "...", "B": "...", "C": "...", "D": "..."},
    source_folder="folder_name"
)
```

In [ ]:
# if __name__ == "__main__":
#     try:
#         main()
#     except KeyboardInterrupt:
#         print("\n\n⚠️  Đã dừng bởi người dùng (Ctrl+C)")
#     except Exception as e:
#         print(f"\n\n❌ Lỗi không mong đợi: {e}")
#         import traceback
#         # http://localhost:8000/api/v1/search/
#         traceback.print_exc()

In [ ]:
# import pandas as pd
# from pathlib import Path

# DATA_DIR = Path("data")
# OUTPUT_FILE = DATA_DIR / "answers_mcq_smallthinker.csv"
# # load results_df from OUTPUT_FILE
# results_df = pd.read_csv(
#     OUTPUT_FILE,
#     on_bad_lines='skip',  # Skip lines with too many fields (pandas >= 1.3.0)
#     # Alternatively, for older pandas: error_bad_lines=False
#     sep=',',  # Explicitly set delimiter if not comma
#     quotechar='"',  # Ensure quotes are handled
#     engine='python'  # Use Python parser for complex cases (slower but more flexible)
# )

# display_summary(results_df)

# print_header("✅ HOÀN TẤT!")
# print(f"\n📁 Kết quả đã được lưu tại: {OUTPUT_FILE.absolute()}")
# print(f"📅 Kết thúc: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
# print("\n" + "=" * 80 + "\n")

### Test Llama.cpp API

In [ ]:
# # Test llama.cpp server API
# import requests
# import json

# # Test configuration
# API_URL = 'http://127.0.0.1:8080/v1/chat/completions'
# TEST_QUESTION = "What is the capital of France?"
# TEST_OPTIONS = {
#     "A": "London",
#     "B": "Paris",
#     "C": "Berlin",
#     "D": "Madrid"
# }

# # Build test prompt
# prompt = f"""Answer this multiple choice question:

# Question: {TEST_QUESTION}

# Options:
# A. {TEST_OPTIONS['A']}
# B. {TEST_OPTIONS['B']}
# C. {TEST_OPTIONS['C']}
# D. {TEST_OPTIONS['D']}

# Provide your answer in this format:
# ANSWER: [letter]
# REASONING: [brief explanation]
# CONFIDENCE: [high/medium/low]
# """

# # Prepare request payload
# payload = {
#     "model": "gpt-like-model",
#     "messages": [
#         {"role": "user", "content": prompt}
#     ],
#     "max_tokens": 200,
#     "temperature": 0.2
# }

# print("🔍 Testing llama.cpp server API...")
# print(f"📡 URL: {API_URL}")
# print(f"❓ Question: {TEST_QUESTION}\n")

# try:
#     # Send request
#     response = requests.post(API_URL, json=payload, timeout=30)
#     response.raise_for_status()

#     # Parse response
#     result = response.json()

#     # Extract answer
#     if "choices" in result and len(result["choices"]) > 0:
#         answer_content = result["choices"][0]["message"]["content"]

#         print("✅ Server Response:")
#         print("─" * 80)
#         print(answer_content)
#         print("─" * 80)

#         # Show usage stats if available
#         if "usage" in result:
#             usage = result["usage"]
#             print(f"\n📊 Usage Stats:")
#             print(f"  • Prompt tokens: {usage.get('prompt_tokens', 'N/A')}")
#             print(f"  • Completion tokens: {usage.get('completion_tokens', 'N/A')}")
#             print(f"  • Total tokens: {usage.get('total_tokens', 'N/A')}")

#         # Show timing if available
#         if "timings" in result:
#             timings = result["timings"]
#             print(f"\n⏱️  Timing:")
#             print(f"  • Prompt: {timings.get('prompt_ms', 0):.2f}ms")
#             print(f"  • Predicted: {timings.get('predicted_ms', 0):.2f}ms")
#             print(f"  • Speed: {timings.get('predicted_per_second', 0):.2f} tokens/sec")
#     else:
#         print("❌ No choices in response")
#         print(json.dumps(result, indent=2))

# except requests.exceptions.ConnectionError:
#     print("❌ Connection Error: Cannot connect to server")
#     print("   Make sure llama-server is running on port 8080")

# except requests.exceptions.Timeout:
#     print("❌ Request timed out")

# except requests.exceptions.HTTPError as e:
#     print(f"❌ HTTP Error: {e}")
#     print(f"   Response: {response.text}")

# except Exception as e:
#     print(f"❌ Unexpected error: {e}")
#     import traceback
#     traceback.print_exc()

## Test OllamaCppClient connect to Llama.cpp Directly

In [ ]:
# # Test OllamaCppClient directly
# print("🧪 Testing OllamaCppClient...")

# test_question = "What is the capital of France?"
# test_options = {
#     "A": "London",
#     "B": "Paris",
#     "C": "Berlin",
#     "D": "Madrid"
# }

# print(f"\n❓ Question: {test_question}")
# print(f"📋 Options: {test_options}\n")

# # Test without context
# print("=" * 80)
# print("Test 1: Direct generation (no context)")
# print("=" * 80)

# result = ask_single_question_cpp(
#     question=test_question,
#     options=test_options,
#     context_chunks=None
# )

# print(f"\n✅ Result:")
# print(f"  Answer: {result.get('predicted_option')}")
# print(f"  Confidence: {result.get('confidence')}")
# print(f"  Reasoning: {result.get('reasoning')}")
# print(f"  Model: {result.get('model')}")

# # Test with empty context (same as None)
# print("\n" + "=" * 80)
# print("Test 2: With empty context list")
# print("=" * 80)

# result2 = ask_single_question_cpp(
#     question=test_question,
#     options=test_options,
#     context_chunks=[]
# )

# print(f"\n✅ Result:")
# print(f"  Answer: {result2.get('predicted_option')}")
# print(f"  Confidence: {result2.get('confidence')}")
# print(f"  Reasoning: {result2.get('reasoning')}")

# print("\n✅ Tests completed!")